In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [60]:
project = "beneaththewater"
if os.path.isdir(project):
  !rm -rf {project}
!git clone https://github.com/hzxsunshine/beneaththewater.git

Cloning into 'beneaththewater'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 98 (delta 39), reused 70 (delta 24), pack-reused 0
Receiving objects: 100% (98/98), 103.84 KiB | 2.00 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [4]:
import beneaththewater

In [5]:
from beneaththewater.utils.data_preparation import get_data, setup_colab_env
get_data()
setup_colab_env()
!find . -type f -name '._*' -delete

Downloading...
From: https://drive.google.com/uc?id=1Z5LKgmZmFRCQ6kZzHWTcV8gzeLcxHtXE
To: /content/uieb_dataset.tar.gz
100%|██████████| 1.52G/1.52G [00:12<00:00, 124MB/s]


Downloaded data from Google Drive, start unzipping...
Unzipped data.
Installing dependencies...
Installing CLIP...
Finished.


In [61]:
from beneaththewater.configs.constants import path_to_data
from beneaththewater.utils.data_preparation import prepare_the_dataframe, prepare_challenge_dataframe
uieb_df = prepare_the_dataframe(path_to_data)
uieb_challenging_df = prepare_challenge_dataframe(path_to_data)

Number of files in the first directory:  890
Number of files in the second directory:  890
Number of common files:  890
Example of common files:  ['191_img_.png', '593_img_.png', '678_img_.png', '536_img_.png', '584_img_.png']
Number of files in the directory:  60
Example of files:  ['7277.png', '2.png', '616.png', '3201.png', '6527.png']


In [27]:
uieb_df.head()

,name,raw_path,reference_path
0,191_img_.png,./uieb_dataset/raw-890/191_img_.png,./uieb_dataset/reference-890/191_img_.png
1,593_img_.png,./uieb_dataset/raw-890/593_img_.png,./uieb_dataset/reference-890/593_img_.png
2,678_img_.png,./uieb_dataset/raw-890/678_img_.png,./uieb_dataset/reference-890/678_img_.png
3,536_img_.png,./uieb_dataset/raw-890/536_img_.png,./uieb_dataset/reference-890/536_img_.png
4,584_img_.png,./uieb_dataset/raw-890/584_img_.png,./uieb_dataset/reference-890/584_img_.png


In [7]:
import torch
import clip
from PIL import Image

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 143MiB/s]


In [62]:
from beneaththewater.configs.constants import select_contrastive_pairs
contrastive_pairs = select_contrastive_pairs(indices=None)
print(f"Current Pairs are: ", contrastive_pairs)

Current Pairs are:  [['Accurate Color Representation', 'Blue/Green Color Cast'], ['Vibrant and vivid', 'Dull and washed-out'], ['Richly detailed', ' Blurry and indistinct'], ['Well-Lit Underwater', 'Underlit Underwater'], ['Crystal-clear', 'Obscured and murky'], ['Balanced Exposure', 'Backlit Subjects'], ['Sharp Aquatic Details', 'Blurred Movement'], ['Natural Ambient Light', 'Artificial Light']]


In [63]:
from beneaththewater.utils.get_clip_processor import process

In [64]:
raw_df = uieb_df[['name', 'raw_path']]
reference_df = uieb_df[['name', 'reference_path']]

In [65]:
raw_df = process(raw_df, contrastive_pairs)
reference_df = process(reference_df, contrastive_pairs)

['raw_path']


100%|██████████| 890/890 [00:53<00:00, 16.68it/s]


['reference_path']


100%|██████████| 890/890 [00:54<00:00, 16.35it/s]


In [68]:
raw_df.to_csv("raw_ground_truth.csv")

In [69]:
reference_df.to_csv("reference_ground_truth.csv")